# Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

C:\Users\Predator\Envs\kn_ml\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\Predator\Envs\kn_ml\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


# Create Fully Connected Network

In [2]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN,self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
    
    
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model = NN(784,10)
x = torch.randn(64,784)
print(model(x).shape)

torch.Size([64, 10])


# Set Device

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
# BOILDER PLATE CODE FOR TRAINING AND EVALUATING OUT MODEL IN PYTORCH

# Change these values if you want the training to run quicker or slower.
EPOCH_SIZE = 512
TEST_SIZE = 256

def train(model, optimizer, train_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()


def test(model, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    return correct / total

# Setting Up Tune

Here, we define a function that trains the Pytorch model for multiple epochs. The function will be executed on a separate **Ray Actor (process)** underneath the hood, so we need to communicate the performance of the model back to Tune.

To do this, we call `**tune.report**` in our training function, which sends the performance value back to Tun.e

In [10]:
train_dataset = datasets.MNIST(root='dataset/', transform = transforms.ToTensor(),download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size = 64, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', transform = transforms.ToTensor(),download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size = 64, shuffle=True)


In [17]:
def train_nn(config):
    #Setting up Data
    train_dataset = datasets.MNIST(root='dataset/', transform = transforms.ToTensor(),download=True)
    train_loader = DataLoader(dataset=train_dataset, batch_size = 64, shuffle=True)
    
    test_dataset = datasets.MNIST(root='dataset/', transform = transforms.ToTensor(),download=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size = 64, shuffle=True)

    
    #Setting up device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    model = NN()
    model.to(device)
    
    optimizer = optim.SGD(
        model.parameters(), lr=config['lr'], 
        momentum = config['momentum']
    )
    
    for i in range(10):
        train(model, optimizer, train_loader)
        acc = test(model, test_loader)
        
        #send the current training result back to Tune
        tune.report(mean_accuracy = acc)
        
        if i % 5 == 0:
            # This saves the model to the trial directory
            torch.save(model.state_dict(), "./model.pth")

In [24]:
import ray

In [25]:
search_space = {
    "lr": tune.sample_from(lambda spec: 10**(-10 * np.random.rand())),
    "momentum": tune.uniform(0.1, 0.9)
}

# datasets.MNIST("~/data", train=True, download=True)
ray.init()
analysis = tune.run(train_nn, config=search_space)

FileNotFoundError: [WinError 2] The system cannot find the file specified